In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
file_path = "C:/Users/emred/OneDrive/Masaüstü/Docs/3rd Year/Spring/CENG404 - Special Topics in CENG"

### Loading Datas

In [4]:
ground_truth_df = pd.read_csv(file_path + '/Doktor Raporları/data' + '/mimic-cxr-2.1.0-test-set-labeled.csv')



results_df = pd.read_json(file_path + '/merged.json')
labels_df = pd.json_normalize(results_df['labels'])
results_df = pd.concat([results_df.drop(columns=['labels']), labels_df], axis = 1)



results_df = results_df.replace({
        'Yes' : 1,
        'No' : 0,
        'Maybe' : -1,
        'Undefined' : np.nan
    }
)



results_df['patient_id'] = results_df["patient_id"].str.replace("p", "").astype(int)
results_df['report_name'] = results_df['report_name'].str.replace('s', '').str.replace('.txt', '').astype(int)
results_df = results_df.rename(columns = {
    'patient_id' : 'subject_id',
    'report_name' : 'study_id'
})



chexpert_results_df = pd.read_csv(file_path + '/comparison_relevant.csv')
ground_truth_df = ground_truth_df.drop('No Finding', axis = 1)
results_df = results_df.drop('subject_id', axis = 1)
chexpert_results_df = chexpert_results_df.drop('No Finding', axis = 1)



# Fill NaN values w/ 0
ground_truth_df = ground_truth_df.fillna(0)
results_df = results_df.fillna(0)
chexpert_results_df = chexpert_results_df.fillna(0)



ground_truth_df = ground_truth_df.rename(columns = {'Airspace Opacity' : 'Lung Opacity'})
reordered_columns = [col for col in ground_truth_df.columns]
results_df = results_df[reordered_columns]
chexpert_results_df = chexpert_results_df[reordered_columns]

C:\Users\emred\AppData\Local\Temp\ipykernel_21344\94410593.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df = results_df.replace({


### Find which reports are missing in results_df, and save them as a json file.

In [6]:
results_df = results_df.drop_duplicates()

ground_truth_ids = {id for id in ground_truth_df['study_id']}

results_ids = {id for id in results_df['study_id']}

missing_ids = {id for id in ground_truth_ids if id not in results_ids}

with open("relevant_reports.json", "r") as json_file:
    data = json.load(json_file)

filtered_data = [entry for entry in data if int(entry['study_id'].replace("s", "")) in missing_ids]

output_file = "filtered_data.json"
with open(output_file, "w") as json_file:
    json.dump(filtered_data, json_file, indent=4)

print(f'File {output_file} is saved.')

File filtered_data.json is saved.


### Element-wise comparison

In [8]:
# matched_df = results_df.merge(ground_truth_df, on = ['subject_id', 'study_id'], suffixes = ('_results', '_ground_truth'))
matched_df = results_df.merge(ground_truth_df, on = ['study_id'], suffixes = ('_results', '_ground_truth'))

In [9]:
merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))

results = []

columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]
total = len(results_df) 

for col in columns_to_compare:
    matches = (merged[f"{col}_results"] == merged[f"{col}_ground_truth"]).sum()
    accuracy = matches / total * 100 
    results.append({'Category': col, 'Total': total, 'Matches': matches, 'Accuracy': f"{accuracy:.2f}%"})

accuracy_df = pd.DataFrame(results)

accuracy_df

,Category,Total,Matches,Accuracy
0,Enlarged Cardiomediastinum,442,395,89.37%
1,Cardiomegaly,442,379,85.75%
2,Lung Lesion,442,423,95.70%
3,Lung Opacity,442,365,82.58%
4,Edema,442,399,90.27%
5,Consolidation,442,425,96.15%
6,Pneumonia,442,419,94.80%
7,Atelectasis,442,396,89.59%
8,Pneumothorax,442,437,98.87%
9,Pleural Effusion,442,405,91.63%


In [10]:
# Merge the results and ground truth dataframes
merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))

# Initialize a list to store results
results = []

# Columns to compare (excluding 'study_id')
columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]

# Iterate through each column to calculate metrics
for col in columns_to_compare:
    # Get the predictions and ground truth for the current column
    y_pred = merged[f"{col}_results"]
    y_true = merged[f"{col}_ground_truth"]

    # Filter out rows where either y_pred or y_true is NaN (Undefined)
    valid_indices = ~y_pred.isna() & ~y_true.isna()
    y_pred = y_pred[valid_indices]
    y_true = y_true[valid_indices]

    # Total valid rows
    total_valid = len(y_true)

    # Calculate matches for accuracy
    matches = (y_pred == y_true).sum()
    accuracy = (matches / total_valid) * 100 if total_valid > 0 else 0

    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN) for each class
    tp = ((y_pred == 1) & (y_true == 1)).sum()  # True Positives for "Yes"
    fp = ((y_pred == 1) & (y_true != 1)).sum()  # False Positives for "Yes"
    fn = ((y_pred != 1) & (y_true == 1)).sum()  # False Negatives for "Yes"

    # Precision, Recall, and F1-Score for "Yes" class
    precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
    recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0

    # Append results for the current column
    results.append({
        'Category': col,
        'Total Valid': total_valid,
        'Matches': matches,
        'Accuracy': f"{accuracy:.2f}%",
        'Precision': f"{precision:.2f}%",
        'Recall': f"{recall:.2f}%",
        'F1-Score': f"{f1_score:.2f}%"
    })

# Create a DataFrame to display the results
accuracy_df = pd.DataFrame(results)

# Display the resulting DataFrame
accuracy_df

,Category,Total Valid,Matches,Accuracy,Precision,Recall,F1-Score
0,Enlarged Cardiomediastinum,442,395,89.37%,68.42%,37.14%,48.15%
1,Cardiomegaly,442,379,85.75%,75.44%,88.66%,81.52%
2,Lung Lesion,442,423,95.70%,73.91%,91.89%,81.93%
3,Lung Opacity,442,365,82.58%,63.19%,93.64%,75.46%
4,Edema,442,399,90.27%,84.21%,93.02%,88.40%
5,Consolidation,442,425,96.15%,75.93%,95.35%,84.54%
6,Pneumonia,442,419,94.80%,87.50%,83.33%,85.37%
7,Atelectasis,442,396,89.59%,75.86%,95.65%,84.62%
8,Pneumothorax,442,437,98.87%,93.10%,90.00%,91.53%
9,Pleural Effusion,442,405,91.63%,91.18%,91.18%,91.18%


In [11]:
# Merge the two DataFrames on 'study_id'
merged_df = pd.merge(results_df, chexpert_results_df, on='study_id', suffixes=('_gemini', '_chexpert'))

# Initialize a list to store correlation results
correlation_results = []

# Columns to compare (excluding 'study_id')
columns_to_compare = [col for col in results_df.drop('study_id', axis = 1).columns];

# Iterate through each category to calculate correlation
for col in columns_to_compare:
    # Get predictions from both models for the current category
    model1_preds = merged_df[f"{col}_gemini"]
    model2_preds = merged_df[f"{col}_chexpert"]

    # Handle NaN values (e.g., exclude rows with NaN in either model)
    valid_indices = (~model1_preds.isna()) & (~model2_preds.isna())
    model1_valid = model1_preds[valid_indices]
    model2_valid = model2_preds[valid_indices]

    # Calculate correlation (Pearson)
    if len(model1_valid) > 1:  # Ensure there are enough data points
        correlation = model1_valid.corr(model2_valid)
    else:
        correlation = None  # Not enough data to calculate correlation

    # Append results for the current category
    correlation_results.append({
        'Category': col,
        'Correlation': correlation
    })

# Create a DataFrame to display the results
correlation_df = pd.DataFrame(correlation_results)

# Display the resulting DataFrame
correlation_df

,Category,Correlation
0,Enlarged Cardiomediastinum,0.083974
1,Cardiomegaly,0.487436
2,Lung Lesion,0.529702
3,Lung Opacity,0.745204
4,Edema,0.614469
5,Consolidation,0.595163
6,Pneumonia,0.140462
7,Atelectasis,0.782207
8,Pneumothorax,0.467113
9,Pleural Effusion,0.669535


In [12]:
merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))

results = []

columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]
total = len(results_df) 

for col in columns_to_compare:
    matches = (merged[f"{col}_results"] == merged[f"{col}_ground_truth"]).sum()
    accuracy = matches / total * 100 
    results.append({'Category': col, 'Total': total, 'Matches': matches, 'Accuracy': f"{accuracy:.2f}%"})

accuracy_df = pd.DataFrame(results)

In [13]:
accuracy_df

,Category,Total,Matches,Accuracy
0,Enlarged Cardiomediastinum,442,395,89.37%
1,Cardiomegaly,442,379,85.75%
2,Lung Lesion,442,423,95.70%
3,Lung Opacity,442,365,82.58%
4,Edema,442,399,90.27%
5,Consolidation,442,425,96.15%
6,Pneumonia,442,419,94.80%
7,Atelectasis,442,396,89.59%
8,Pneumothorax,442,437,98.87%
9,Pleural Effusion,442,405,91.63%


In [14]:
# Merge the results and ground truth dataframes
merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))

# Initialize a list to store results
results = []

# Columns to compare (excluding 'study_id')
columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]

# Iterate through each column to calculate metrics
for col in columns_to_compare:
    # Get the predictions and ground truth for the current column
    y_pred = merged[f"{col}_results"]
    y_true = merged[f"{col}_ground_truth"]

    # Filter out rows where either y_pred or y_true is NaN (Undefined)
    valid_indices = ~y_pred.isna() & ~y_true.isna()
    y_pred = y_pred[valid_indices]
    y_true = y_true[valid_indices]

    # Total valid rows
    total_valid = len(y_true)

    # Calculate matches for accuracy
    matches = (y_pred == y_true).sum()
    accuracy = (matches / total_valid) * 100 if total_valid > 0 else 0

    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN) for each class
    tp = ((y_pred == 1) & (y_true == 1)).sum()  # True Positives for "Yes"
    fp = ((y_pred == 1) & (y_true != 1)).sum()  # False Positives for "Yes"
    fn = ((y_pred != 1) & (y_true == 1)).sum()  # False Negatives for "Yes"

    # Precision, Recall, and F1-Score for "Yes" class
    precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
    recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0

    # Append results for the current column
    results.append({
        'Category': col,
        'Total Valid': total_valid,
        'Matches': matches,
        'Accuracy': f"{accuracy:.2f}%",
        'Precision': f"{precision:.2f}%",
        'Recall': f"{recall:.2f}%",
        'F1-Score': f"{f1_score:.2f}%"
    })

# Create a DataFrame to display the results
accuracy_df = pd.DataFrame(results)

# Display the resulting DataFrame
accuracy_df

,Category,Total Valid,Matches,Accuracy,Precision,Recall,F1-Score
0,Enlarged Cardiomediastinum,442,395,89.37%,68.42%,37.14%,48.15%
1,Cardiomegaly,442,379,85.75%,75.44%,88.66%,81.52%
2,Lung Lesion,442,423,95.70%,73.91%,91.89%,81.93%
3,Lung Opacity,442,365,82.58%,63.19%,93.64%,75.46%
4,Edema,442,399,90.27%,84.21%,93.02%,88.40%
5,Consolidation,442,425,96.15%,75.93%,95.35%,84.54%
6,Pneumonia,442,419,94.80%,87.50%,83.33%,85.37%
7,Atelectasis,442,396,89.59%,75.86%,95.65%,84.62%
8,Pneumothorax,442,437,98.87%,93.10%,90.00%,91.53%
9,Pleural Effusion,442,405,91.63%,91.18%,91.18%,91.18%


In [15]:
# Merge the two DataFrames on 'study_id'
merged_df = pd.merge(results_df, chexpert_results_df, on='study_id', suffixes=('_gemini', '_chexpert'))

# Initialize a list to store correlation results
correlation_results = []

# Columns to compare (excluding 'study_id')
columns_to_compare = [col for col in results_df.drop('study_id', axis = 1).columns];

# Iterate through each category to calculate correlation
for col in columns_to_compare:
    # Get predictions from both models for the current category
    model1_preds = merged_df[f"{col}_gemini"]
    model2_preds = merged_df[f"{col}_chexpert"]

    # Handle NaN values (e.g., exclude rows with NaN in either model)
    valid_indices = (~model1_preds.isna()) & (~model2_preds.isna())
    model1_valid = model1_preds[valid_indices]
    model2_valid = model2_preds[valid_indices]

    # Calculate correlation (Pearson)
    if len(model1_valid) > 1:  # Ensure there are enough data points
        correlation = model1_valid.corr(model2_valid)
    else:
        correlation = None  # Not enough data to calculate correlation

    # Append results for the current category
    correlation_results.append({
        'Category': col,
        'Correlation': correlation
    })

# Create a DataFrame to display the results
correlation_df = pd.DataFrame(correlation_results)

# Display the resulting DataFrame
correlation_df

,Category,Correlation
0,Enlarged Cardiomediastinum,0.083974
1,Cardiomegaly,0.487436
2,Lung Lesion,0.529702
3,Lung Opacity,0.745204
4,Edema,0.614469
5,Consolidation,0.595163
6,Pneumonia,0.140462
7,Atelectasis,0.782207
8,Pneumothorax,0.467113
9,Pleural Effusion,0.669535


### This time, I evaluate based on 1's and 0's

In [17]:
# Filter out rows where predictions or ground truth are -1 or NaN
copied_merged = merged.copy()

# Iterate through each category to calculate metrics
results = []
categories = [col.replace("_results", "") for col in merged.columns if "_results" in col]

for category in categories:
    # Filter valid rows for the current category
    y_pred = copied_merged[f"{category}_results"]
    y_true = copied_merged[f"{category}_ground_truth"]
    valid_indices = (y_pred.isin([0, 1])) & (y_true.isin([0, 1]))
    
    y_pred = y_pred[valid_indices]
    y_true = y_true[valid_indices]
    
    # Total valid rows
    total_valid = len(y_true)
    
    # Calculate matches for accuracy
    matches = (y_pred == y_true).sum()
    accuracy = (matches / total_valid) * 100 if total_valid > 0 else 0
    
    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
    tp = ((y_pred == 1) & (y_true == 1)).sum()
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    fn = ((y_pred == 0) & (y_true == 1)).sum()
    
    # Precision, Recall, and F1-Score
    precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
    recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
    
    # Append results for the current category
    results.append({
        'Category': category,
        'Total Valid': total_valid,
        'Matches': matches,
        'Accuracy': f"{accuracy:.2f}%",
        'Precision': f"{precision:.2f}%",
        'Recall': f"{recall:.2f}%",
        'F1-Score': f"{f1_score:.2f}%"
    })

# Create a DataFrame to display the results
filtered_metrics_df = pd.DataFrame(results)

# Display the resulting DataFrame
filtered_metrics_df

,Category,Total Valid,Matches,Accuracy,Precision,Recall,F1-Score
0,Enlarged Cardiomediastinum,414,393,94.93%,68.42%,46.43%,55.32%
1,Cardiomegaly,403,375,93.05%,77.48%,96.63%,86.00%
2,Lung Lesion,432,421,97.45%,79.07%,94.44%,86.08%
3,Lung Opacity,431,365,84.69%,63.19%,94.50%,75.74%
4,Edema,406,392,96.55%,89.89%,94.12%,91.95%
5,Consolidation,428,418,97.66%,83.67%,95.35%,89.13%
6,Pneumonia,378,376,99.47%,97.22%,97.22%,97.22%
7,Atelectasis,407,382,93.86%,82.09%,99.10%,89.80%
8,Pneumothorax,433,432,99.77%,100.00%,96.43%,98.18%
9,Pleural Effusion,406,394,97.04%,94.51%,98.10%,96.27%


In [18]:
# Merge the two DataFrames on 'study_id'
merged_df = pd.merge(results_df, chexpert_results_df, on='study_id', suffixes=('_results', '_chexpert'))

# Initialize a list to store correlation results
correlation_results = []

# Columns to compare (excluding 'study_id')
columns_to_compare = [col for col in results_df.columns if col != 'study_id']

# Iterate through each category to calculate correlation
for col in columns_to_compare:
    # Get predictions from both models for the current category
    model1_preds = merged_df[f"{col}_results"]
    model2_preds = merged_df[f"{col}_chexpert"]
    
    # Filter valid rows (exclude -1 and NaN)
    valid_indices = (model1_preds.isin([0, 1])) & (model2_preds.isin([0, 1]))
    model1_valid = model1_preds[valid_indices]
    model2_valid = model2_preds[valid_indices]
    
    # Calculate correlation (Pearson)
    if len(model1_valid) > 1:  # Ensure there are enough data points
        correlation = model1_valid.corr(model2_valid)
    else:
        correlation = None  # Not enough data to calculate correlation
    
    # Append results for the current category
    correlation_results.append({
        'Category': col,
        'Correlation': correlation
    })

# Create a DataFrame to display the results
correlation_df = pd.DataFrame(correlation_results)

# Display the resulting DataFrame
correlation_df

,Category,Correlation
0,Enlarged Cardiomediastinum,0.379531
1,Cardiomegaly,0.847563
2,Lung Lesion,0.778915
3,Lung Opacity,0.880441
4,Edema,0.855382
5,Consolidation,0.965691
6,Pneumonia,0.761762
7,Atelectasis,0.950771
8,Pneumothorax,0.797318
9,Pleural Effusion,0.974233


In [19]:
len(ground_truth_df)

687

In [20]:
maybe_counts = (ground_truth_df.drop(['study_id'], axis=1) == -1).sum()

maybe_percentages = (maybe_counts / len(ground_truth_df)) * 100

maybe_summary_df = pd.DataFrame({
    '-1 Count': maybe_counts,
    '-1 Percentage': maybe_percentages
})

maybe_summary_df

,-1 Count,-1 Percentage
Enlarged Cardiomediastinum,38,5.531295
Cardiomegaly,39,5.676856
Lung Lesion,9,1.310044
Lung Opacity,0,0.000000
Edema,41,5.967977
Consolidation,26,3.784571
Pneumonia,89,12.954876
Atelectasis,41,5.967977
Pneumothorax,10,1.455604
Pleural Effusion,23,3.347889


In [21]:
len(results_df)

442

In [22]:
len(ground_truth_df)

687

In [23]:
len(missing_ids)

245